In [15]:
import pymysql #mysql 연동
import numpy as np #숫자 배열&계산
from scipy.spatial.distance import cosine #코사인 유사도 계산

#user_record와 album 테이블에서 데이터 가져오는 함수
def fetch_records_and_albums(connection):
    #db연결할 때 커서를 생성하고, sql쿼리를 실행한 후 블록 끝나면 커서가 자동으로 닫힘
    with connection.cursor() as cursor:
        #user_record 테이블에서 user_id와 album_id 가져옴
        cursor.execute("SELECT user_id, album_id FROM user_record")
        records = cursor.fetchall()
        #album 테이블에서 id와 크롤링한 필드 값 가져옴
        cursor.execute("""
            SELECT id, music_acousticness, music_danceability, music_energy, 
                   music_liveness, music_loudness,
                   music_tempo, music_valence 
            FROM album
        """)
        albums = cursor.fetchall()
    #결과값 튜플로 반환
    return records, albums

# 사용자가 재생한 음악들의 평균 특성 벡터 계산
#album_dict: 앨범id를 키로 하고, 특성 벡터를 값으로 가지는 딕셔너리
#feature_indices: 평균을 계산할 특성 벡터의 인덱스 리스트
def calculate_average_features(user_records, album_dict, feature_indices):
    # 첫 번째 앨범의 특성 벡터 길이를 기준으로 초기화
    first_album_id = next(iter(album_dict.keys()))
    feature_vector_length = album_dict[first_album_id].shape[0]
    
    # 평균 특성 벡터 및 앨범 수 초기화
    feature_sums = np.zeros(feature_vector_length)
    count = 0

    # 사용자가 재생한 앨범의 특성 벡터를 합산
    for user_id, album_id in user_records:
        if album_id in album_dict:
            feature_vector = album_dict[album_id]
            # feature_indices가 특성 벡터 길이를 초과하지 않도록 확인
            if all(idx < feature_vector_length for idx in feature_indices):
                feature_sums += feature_vector[feature_indices]
                count += 1

    # 사용자가 앨범을 재생하지 않은 경우 0으로 반환
    if count == 0:
        return feature_sums
    else:
        return feature_sums / count



#main 함수
def main():
    #mysql 연결
    connection = pymysql.connect(
        host='mitidb.cvm64ss6y2xv.ap-northeast-2.rds.amazonaws.com',       
        user='minseo',  
        password='Alstj!!809', 
        database='mitiDB'
    )

    # features_group1과 features_group2의 인덱스 정의
    # features_group1과 features_group2의 인덱스 정의 (실제 데이터베이스의 열 구조에 맞춰 수정)
    features_group1_indices = [2, 3, 5, 6, 7]  # danceability, energy, tempo, loudness, valence
    features_group2_indices = [1, 4]           # acousticness, liveness



    try:
        #user_record, album 테이블에서 데이터 가져옴
        records, albums = fetch_records_and_albums(connection)

        #album_id를 키로 하고,앨범 특성을 값으로 하는 딕셔너리 생성
        album_dict = {album[0]: np.array(album[1:]) for album in albums}

        #user_record에 있는 모든 user_id에 대해 평균 특성 벡터 계산 (두 그룹으로 나눠서 진행)
        average_features_group1 = calculate_average_features(records, album_dict, features_group1_indices)
        average_features_group2 = calculate_average_features(records, album_dict, features_group2_indices)

        #각 앨범 특성 벡터와 사용자 평균 특성 벡터 간 코사인 유사도 계산 후 리스트에 저장
        similarities_group1 = []
        similarities_group2 = []
        
        for album_id, features in album_dict.items():
            #코사인 유사도 계산
            similarity_group1 = 1 - cosine(average_features_group1, features[features_group1_indices])
            similarity_group2 = 1 - cosine(average_features_group2, features[features_group2_indices])
            #앨범 id, 코사인 유사도 쌍으로 구성하여 저장
            similarities_group1.append((album_id, similarity_group1))
            similarities_group2.append((album_id, similarity_group2))

        #두 그룹을 합친 후 유사도 내림차순으로 정렬
        combined_similarities = similarities_group1 + similarities_group2
        sorted_albums = sorted(combined_similarities, key=lambda x: x[1], reverse=True)

        # 중복을 제거하면서 상위 20개 앨범을 선택
        top_albums = [] #최종 상위 20개
        seen_album_ids = set()  #중복 확인을 위한

        for album_id, similarity in sorted_albums:
            # 이미 선택된 앨범인지 확인
            if album_id not in seen_album_ids:
                seen_album_ids.add(album_id)
                top_albums.append((album_id, similarity))

                # 상위 20개 앨범을 모두 선택했으면 종료
                if len(top_albums) == 20:
                    break

        # 결과 customized_rec 테이블에 삽입
        with connection.cursor() as cursor:
            for user_id, _ in records:
                for album_id, _ in top_albums:
                    try:
                        # 중복 삽입 방지를 위해 IntegrityError 처리
                        cursor.execute(
                            "INSERT INTO customized_rec (user_id, album_id) VALUES (%s, %s)",
                            (user_id, album_id)
                        )
                    except pymysql.IntegrityError:
                        continue

            # 변경사항을 데이터베이스에 반영
            connection.commit()

        # 완료 메시지 출력
        print("Top 20 albums by cosine similarity have been inserted into customized_rec table.")

    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        #데이터베이스 연결 닫기
        connection.close()  

if __name__ == "__main__":
    main()


ValueError: operands could not be broadcast together with shapes (7,) (2,) (7,) 